In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PREPROCESSING

Functions to load and cleanup the data for training with Flair.



In [ ]:
import csv
import re
import sys
import pandas as pd

# load data
def load_data(fnames):
    data = []
    for fname in fnames:
        data.append(pd.read_csv(fname, sep='\t', encoding='utf-8'))
    data = pd.concat(data)
    targets = set(data['Target'])
    return data, list(targets)

# pre-process tweets
def cleanup(tweet):
    """we remove urls, hashtags and user symbols"""
    tweet = re.sub(r"http\S+", "", tweet.replace("#", "").replace("@", "").replace('\n', ' ').replace('\t', ' '))
    return tweet

def format_label_to_flair(label):
  return label


# Loading and Pre-processing the datasets


We load and cleanup the data using the functions defined above.



## ASSIGNMENT 1

+ TODO: Write the format_label_to_flair(label) function above to obtain the label column in the format shown below.
+ TODO: Write the required code to load, cleanup and format the labels for both the training and test sets obtaining the output shown below.

In [ ]:
# data path. trial data used as training too.
trial_file = "/content/drive/My Drive/Colab Notebooks/2022-ILTAPP/datasets/stance-semeval2016/semeval2016-task6-trialdata.utf-8.txt"
train_file = "/content/drive/My Drive/Colab Notebooks/2022-ILTAPP/datasets/stance-semeval2016/semeval2016-task6-trainingdata.utf-8.txt"
test_file = "/content/drive/My Drive/Colab Notebooks/2022-ILTAPP/datasets/stance-semeval2016/SemEval2016-Task6-subtaskA-testdata-gold.txt"

# TODO write your code here


display(training_data)

,ID,Target,Tweet,Stance,Clean_tweet
0,1,Hillary Clinton,"@tedcruz And, #HandOverTheServer she wiped cle...",__label__AGAINST,"tedcruz And, HandOverTheServer she wiped clean..."
1,2,Hillary Clinton,Hillary is our best choice if we truly want to...,__label__FAVOR,Hillary is our best choice if we truly want to...
2,3,Hillary Clinton,@TheView I think our country is ready for a fe...,__label__AGAINST,TheView I think our country is ready for a fem...
3,4,Hillary Clinton,I just gave an unhealthy amount of my hard-ear...,__label__AGAINST,I just gave an unhealthy amount of my hard-ear...
4,5,Hillary Clinton,@PortiaABoulger Thank you for adding me to you...,__label__NONE,PortiaABoulger Thank you for adding me to your...
...,...,...,...,...,...
2809,2910,Legalization of Abortion,"There's a law protecting unborn eagles, but no...",__label__AGAINST,"There's a law protecting unborn eagles, but no..."
2810,2911,Legalization of Abortion,I am 1 in 3... I have had an abortion #Abortio...,__label__AGAINST,I am 1 in 3... I have had an abortion Abortion...
2811,2912,Legalization of Abortion,How dare you say my sexual preference is a cho...,__label__AGAINST,How dare you say my sexual preference is a cho...
2812,2913,Legalization of Abortion,"Equal rights for those 'born that way', no rig...",__label__AGAINST,"Equal rights for those 'born that way', no rig..."


## ASSIGNMENT 2

From the training_data and testing_data dataframes obtained in the previous step, perform the following steps:

+ TODO: Iterate over the targets to write a training and test set in the format required by Flair to train a text classifier:

```
__label__AGAINST first_tweet_in_dataframe
__label__NONE second_tweet_in_dataframe
__label__FAVOR third_tweet_in_dataframe
```
*NOTE: One tweet per line*

The result should be a training set and test set in files for each target. For example:



```
train.Feminist Movement.txt
test.Feminist Movement.txt
```

HINT: You just need to use the Stance and Clean_tweet columns from the training_data and test_data dataframes and write them to a csv with "\t" as separator.

https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md#reading-a-text-classification-dataset


In [ ]:
# TODO Write your code here

# INSTALL FLAIR

In [ ]:
!pip install flair==0.8

     |████████████████████████████████| 450kB 16.2MB/s 
     |████████████████████████████████| 798kB 44.3MB/s 
     |████████████████████████████████| 19.7MB 7.0MB/s 
     |████████████████████████████████| 1.1MB 46.8MB/s 
     |████████████████████████████████| 983kB 48.8MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 1.3MB 46.5MB/s 
     |████████████████████████████████| 890kB 46.9MB/s 
     |████████████████████████████████| 2.9MB 50.3MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116678 sha256=2eb456c87315e3a4a56c85be7b473b8b094dfcc46183f3fc972fb2aa2b83208f
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for sqlitedict: filename=sqlitedict-1.7.0-cp36-none-any.whl size=14376 sha256=8243c639e02713aeb540b680e12274549f57c757e3658bb8e3bd314b438549d5
  Stored in directory: /root/.cache/pip/wheels/cf/c6/4f/2c64a43f041415eb8b8740bd80e

# Load the training data created in ASSIGNMENT 2

In [ ]:
from flair.data import Corpus                                                                                                                                                           
from flair.datasets import ClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, FastTextEmbeddings, DocumentRNNEmbeddings                                                             
from flair.models import TextClassifier                                                                                                                                                 
from flair.trainers import ModelTrainer

# get the corpus
corpus_folder = '/content/drive/My Drive/datasets/stance-semeval2016'
corpus: Corpus = ClassificationCorpus(corpus_folder,
                                      train_file='train.Feminist Movement.txt',
                                      #dev_file='train.Feminist Movement.txt',
                                      test_file='test.Feminist Movement.txt'
)

# create the label dictionary
label_dict = corpus.make_label_dictionary()

2021-02-09 12:59:40,438 Reading data from /content/drive/My Drive/datasets/stance-semeval2016
2021-02-09 12:59:40,438 Train: /content/drive/My Drive/datasets/stance-semeval2016/train.Feminist Movement.txt
2021-02-09 12:59:40,443 Dev: None
2021-02-09 12:59:40,444 Test: /content/drive/My Drive/datasets/stance-semeval2016/test.Feminist Movement.txt
2021-02-09 12:59:40,460 Computing label dictionary. Progress:


100%|██████████| 883/883 [00:00<00:00, 1477.42it/s]

2021-02-09 12:59:41,181 [b'FAVOR', b'AGAINST', b'NONE']


# INSTALL Word Embeddings and instantiate the TextClassifier

In [ ]:
# 3. make a list of word embeddings
word_embeddings = [
                   WordEmbeddings('en-crawl'),
                   #FlairEmbeddings('news-forward'),
                   #FlairEmbeddings('news-backward'),
]

# 4. initialize document embedding by passing list of word embeddings                                                                                                                   
# Can choose between many RNN types (GRU by default, to change use rnn_type parameter)                                                                                                  
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   hidden_size=512,
                                                                   reproject_words=True,
                                                                   reproject_words_dimension=256,
                                                                   rnn_type='LSTM',
)
                                                                                                                                                                                         
# 5. create the text classifier
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict)
# 6. initialize the text classifier trainer
trainer = ModelTrainer(classifier, corpus)

2021-02-09 12:59:41,734 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpuwfybcya


100%|██████████| 1200000128/1200000128 [01:03<00:00, 18858067.50B/s]

2021-02-09 13:00:45,698 copying /tmp/tmpuwfybcya to cache at /root/.flair/embeddings/en-fasttext-crawl-300d-1M.vectors.npy


2021-02-09 13:00:49,992 removing temp file /tmp/tmpuwfybcya
2021-02-09 13:00:50,640 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M not found in cache, downloading to /tmp/tmpf2898gwq


100%|██████████| 39323680/39323680 [00:01<00:00, 19687567.17B/s]

2021-02-09 13:00:52,970 copying /tmp/tmpf2898gwq to cache at /root/.flair/embeddings/en-fasttext-crawl-300d-1M


2021-02-09 13:00:53,006 removing temp file /tmp/tmpf2898gwq


# Run the trainer for feminism target

In [ ]:
# 7. start the training
trainer.train('flair-feminism-model',
              train_with_dev=False,
              max_epochs=50)

2021-02-09 13:01:10,857 ----------------------------------------------------------------------------------------------------
2021-02-09 13:01:10,859 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('en-crawl')
    )
    (word_reprojection_map): Linear(in_features=300, out_features=256, bias=True)
    (rnn): LSTM(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=3, bias=True)
  (loss_function): CrossEntropyLoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2021-02-09 13:01:10,862 ----------------------------------------------------------------------------------------------------
2021-02-09 13:01:10,865 Corpus: "Corpus: 598 train + 66 dev + 285 test sentences"
2021-02-09 13:01:10,866 ----------------------------------------------------------------------------------------------------
2021-02-09 13:01

{'dev_loss_history': [1.0328080654144287,
  1.0275135040283203,
  1.0248466730117798,
  1.0328657627105713,
  1.028809905052185,
  1.0088070631027222,
  1.0528422594070435,
  1.0378535985946655,
  0.9797680377960205,
  0.9868075847625732,
  0.975127100944519,
  1.0530335903167725,
  0.9950854182243347,
  0.9674684405326843,
  1.0136098861694336,
  0.9719399213790894,
  0.9718084335327148,
  0.9831948280334473,
  0.967137336730957,
  0.9711171984672546,
  0.9700323939323425,
  0.9747815132141113,
  0.9749042987823486,
  0.9701939821243286,
  0.9752936363220215,
  0.9819273948669434,
  0.9717144966125488,
  0.9633159637451172,
  0.9686483144760132,
  0.9693652987480164,
  0.97027587890625,
  0.9643357992172241,
  0.9698681831359863,
  0.9712992906570435,
  0.9687816500663757,
  0.966253399848938,
  0.9652258157730103,
  0.9672719836235046,
  0.9672948718070984,
  0.9686758518218994,
  0.968494176864624,
  0.9672454595565796,
  0.9679700136184692,
  0.9682784080505371,
  0.968038558959960

# ASSIGNMENT 3 (BONUS)

+ TODO: Train the TextClassifier for the other 4 targets in the SemEval training data using the data files you have created in ASSIGNMENT 2.
+ TODO: Write in a table in this notebook the results obtained for the 5 targets. HINT: check the table in Lab2.